In [30]:
import numpy as np
import geopandas as gpd
from scipy.spatial import cKDTree
from datetime import datetime 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
import pandas as pd
import contextily as cnx
import matplotlib.pyplot as plt
from datetime import date
import re
from rasterstats import zonal_stats
import rasterio
import os
import glob
from tqdm import tqdm
from shapely.geometry import box
import math
from shapely.geometry import LineString

<h2> Pair Matching <h2> 

<h5> Getting footprint pairs within a max distance of 40 m with information from pre and post fire. <h5>

In [31]:
# Folder where your batch files are stored
BATCH_FOLDER = "GEDI_Fire_Dates"
PAIRS_FOLDER = "GEDI_Pairs"
os.makedirs(PAIRS_FOLDER, exist_ok=True)

# Collect all batch files
batch_files = glob.glob(os.path.join(BATCH_FOLDER, "*.gpkg"))
print(f"Found {len(batch_files)} batch files.")


Found 33 batch files.


In [11]:
# Convert comma-separated date string into list of datetime objects.

def parse_fire_dates_str(fire_dates_str):
    """Convert comma-separated date string into list of datetime objects."""
    if not fire_dates_str or fire_dates_str.strip() == "":
        return []
    return [pd.to_datetime(d, errors="coerce") for d in fire_dates_str.split(",") if d]

In [28]:
def get_close_pairs_with_fire(gdf_proj, max_distance=40):

    # Ensure gedi_time is datetime
    gdf_proj = gdf_proj.copy()
    gdf_proj["gedi_time"] = pd.to_datetime(gdf_proj["gedi_time"], errors="coerce")

    # Get centroids of polygons for distance calculations
    centroids = gdf_proj.geometry.centroid
    coords = np.column_stack([centroids.x.values, centroids.y.values])

    # KD-tree
    tree = cKDTree(coords)
    dist_matrix = tree.sparse_distance_matrix(tree, max_distance=max_distance, output_type='coo_matrix')

    close_pairs = [(i, j, d) for i, j, d in zip(dist_matrix.row, dist_matrix.col, dist_matrix.data) if i < j]

    records = []
    for i, j, d in tqdm(close_pairs, desc="Finding footprint pairs"):
        fi = gdf_proj.iloc[i]
        fj = gdf_proj.iloc[j]

        # Observation times
        t_i = pd.to_datetime(fi.gedi_time)
        t_j = pd.to_datetime(fj.gedi_time)

        if pd.isna(t_i) or pd.isna(t_j):
            continue  # skip rows without valid times

        # Determine pre/post roles
        if t_i <= t_j:
            pre, post = fi, fj
            pre_idx, post_idx = i, j
            pre_time, post_time = t_i, t_j
        else:
            pre, post = fj, fi
            pre_idx, post_idx = j, i
            pre_time, post_time = t_j, t_i

        # Parse fire dates for both footprints (use union)
        fire_dates_pre = parse_fire_dates_str(pre.fire_dates_str) if "fire_dates_str" in pre.index else []
        fire_dates_post = parse_fire_dates_str(post.fire_dates_str) if "fire_dates_str" in post.index else []
        all_fire_dates = fire_dates_pre + fire_dates_post

        # Fire strictly between pre and post
        between = [fd for fd in all_fire_dates if pd.notnull(fd) and pre_time < fd <= post_time]

        fire_date = min(between) if between else None
        post_fire_months = None
        if fire_date is not None:
            post_fire_months = (post_time.year - fire_date.year) * 12 + (post_time.month - fire_date.month)

        records.append({
            "index_1": i,
            "index_2": j,
            "distance_m": d,

            "pre_index": pre_idx,
            "post_index": post_idx,

            "time_1": pre_time,
            "time_2": post_time,

            # fire info
            "fire": fire_date is not None,
            "fire_date": fire_date,
            "post_fire_months": post_fire_months,

            # biomass
            "pre_agbd": pre.agbd if "agbd" in pre.index else None,
            "post_agbd": post.agbd if "agbd" in post.index else None,
            "delta_agbd": (pre.agbd - post.agbd) if ("agbd" in pre.index and "agbd" in post.index) else None,

            # geometries (polygons) and visualization line (centroids)
            "pre_geom": pre.geometry,
            "post_geom": post.geometry,
            "pair_line": LineString([pre.geometry.centroid, post.geometry.centroid]),
        })

    pairs_gdf = gpd.GeoDataFrame(records, geometry="pair_line", crs=gdf_proj.crs).sort_values("distance_m")
    return pairs_gdf


In [ ]:
# Choose a single CRS for the final merged file.
# For sharing/interop, EPSG:4326 is fine. If you prefer projected for the final output,

COMMON_CRS = "EPSG:4326"

all_pairs_reproj = []

for f in tqdm(batch_files, desc="Processing batches"):
    try:
        gdf = gpd.read_file(f)
        if gdf.empty:
            print(f"Skipping empty file: {f}")
            continue

        # Project to local UTM for accurate distances
        utm_crs = gdf.estimate_utm_crs()
        gdf_proj = gdf.to_crs(utm_crs)

        # Compute pairs in UTM
        pairs = get_close_pairs_with_fire(gdf_proj, max_distance=40)

        if pairs.empty:
            print(f"No pairs found in: {os.path.basename(f)}")
            continue

        # Track source
        pairs["source_file"] = os.path.basename(f)

        # Save intermediate per-batch pairs (optional, useful for recovery)
        intermediate_out = os.path.join(PAIRS_FOLDER, f"pairs_{os.path.splitext(os.path.basename(f))[0]}.gpkg")
        pairs.to_file(intermediate_out, driver="GPKG")

        # Reproject pairs to a single common CRS for concatenation
        pairs_reproj = pairs.to_crs(COMMON_CRS)
        all_pairs_reproj.append(pairs_reproj)

    except Exception as e:
        print(f"Error processing {f}: {e}")



Processing batches: 100%|██████████| 33/33 [03:08<00:00,  5.72s/it]
C:\Users\maria\AppData\Local\Temp\ipykernel_21436\1596285083.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pairs_all = gpd.GeoDataFrame(pd.concat(all_pairs_reproj, ignore_index=True), crs=COMMON_CRS)


ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'

In [35]:
OUTPUT_FILE = os.path.join(PAIRS_FOLDER, "GEDI_Footprint_Pairs_All.gpkg")

# Collect all .gpkg files in GEDI_Pairs
pair_files = sorted(glob.glob(os.path.join(PAIRS_FOLDER, "*.gpkg")))
print(f"Found {len(pair_files)} pair files.")

all_pairs = []

for f in pair_files:
    gdf = gpd.read_file(f)
    if gdf.empty:
        print(f"Skipping empty file: {f}")
        continue

    # Add source file column for traceability
    gdf["source_file"] = os.path.basename(f)

    # Reproject to a common CRS
    gdf = gdf.to_crs("EPSG:4326")
    all_pairs.append(gdf)

# Concatenate all
if all_pairs:
    pairs_all = gpd.GeoDataFrame(
        pd.concat(all_pairs, ignore_index=True),
        geometry="geometry",   # <-- use the actual geometry column
        crs="EPSG:4326"
    )
    pairs_all.to_file(OUTPUT_FILE, driver="GPKG")
    print(f"✅ Merged {len(pair_files)} files into {OUTPUT_FILE} with {len(pairs_all)} pairs total.")
else:
    print("No pair files found to merge.")



Found 33 pair files.


C:\Users\maria\AppData\Local\Temp\ipykernel_21436\1570473667.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(all_pairs, ignore_index=True),


✅ Merged 33 files into GEDI_Pairs\GEDI_Footprint_Pairs_All.gpkg with 133178 pairs total.
